In [22]:
####
#Author: brandon chiazza
#version 1.0
#references: 
#https://www.programiz.com/python-programming/working-csv-files
#https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket
#https://realpython.com/python-boto3-aws-s3/
#CLI aws s3api create-bucket --bucket my-bucket-name --region us-west-2 --create-bucket-configuration LocationConstraint=us-west-2
#https://robertorocha.info/setting-up-a-selenium-web-scraper-on-aws-lambda-with-python/ 
##

###Load libraries
import awscli
import selenium
import boto3
import pandas as pd
import time

from selenium import webdriver


####SCRAPE THE WEBSITE######
###call the webdriver
browser = webdriver.Chrome(executable_path='/Users/chromedriver.exec')

#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

#identify xpath of location to select element
inputElement = browser.find_element_by_xpath("/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]")
inputElement.send_keys('0')
inputElement1 = browser.find_element_by_xpath("/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]").click()

#identify the table to scrape
table = browser.find_element_by_css_selector('table.Bordered')

print(table)


<selenium.webdriver.remote.webelement.WebElement (session="38af251a30cfd13bd95ba41a51ac2b8c", element="a0de0094-2d0d-40a8-9a93-761ada43c807")>


In [23]:
#####CREATE DATE FRAME#####
#create empty dataframe
df =[]

#loop through dataframe to export table
for row in table.find_elements_by_css_selector('tr'):
      cols = df.append([cell.text for cell in row.find_elements_by_css_selector('td')])


#update dataframe with header 
df = pd.DataFrame(df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])
display(df) #let's have a look at the data before creating the CSV file and loading it into s3 

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,None,None,None,None,None,None
1,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
2,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
3,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
4,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY
5,#WalkAway Foundation,47-15-80,832820906,NFP,CARLSBAD,CA
6,04/11 10:17 PM test,47-13-95,206256427,NFP,ALBANY,NY
7,1/20/21 Action Fund,46-99-13,832210730,NFP,SAN FRANCISCO,CA
8,"10/40 Connections, Inc.",45-70-15,621825230,NFP,HIXSON,TN
9,"1000 Feet Project, Inc",45-00-14,473820859,NFP,NEW YORK,NY


In [24]:
###LOAD THE FILE INTO S3####
# prepare csv file name   
pathname = 's3:/alejandrabucket2/'#specify location of s3:/{my-bucket}/
filename= 'charities_bureau_scrape_' #name of your group
datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
filenames3 = "%s%s%s.csv"%(pathname,filename,datetime) #name of the filepath and csv file

#load file into s3. Pandas actually leverages boto to connect to s3 and can push the file directly into an s3 bucket
df.to_csv(filenames3, header=True, line_terminator='\n') 

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

Successfull uploaded file to location:s3:/alejandrabucket2/charities_bureau_scrape_20210505234109.csv


In [30]:
#call the webdriver
browser = webdriver.Chrome(executable_path='/Users/chromedriver.exec')

#enter the url the webdriver needs
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

inputElement = browser.find_element_by_xpath("/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]")
inputElement.send_keys('0')
inputElement1 = browser.find_element_by_xpath("/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]").click()

table = browser.find_element_by_css_selector('table.Bordered')

#create dataframe
def create_DataFrame(table):

    #empty df
    df =[]
    

    #export table
    for row in table.find_elements_by_css_selector('tr'):
          cols = df.append([cell.text for cell in row.find_elements_by_css_selector('td')])


    #add header
    df = pd.DataFrame(df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])

    #blank row removal
    df = df.iloc[1:]
    
    return df

table_1 = create_DataFrame(table)

for i in range(1,5):
    if i == 2 or i == 4:
        continue
    inputElement_all = browser.find_element_by_xpath("/html/body/div[2]/div/table/tbody/tr/td[3]/div/div/span[2]/a["+str(i)+"]").click()

    table = browser.find_element_by_css_selector('table.Bordered')
    table_2 = create_DataFrame(table)
    table_1 = table_1.append(table_2, ignore_index=True)

display(table_1)

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
1,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
2,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
3,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY
4,#WalkAway Foundation,47-15-80,832820906,NFP,CARLSBAD,CA
5,04/11 10:17 PM test,47-13-95,206256427,NFP,ALBANY,NY
6,1/20/21 Action Fund,46-99-13,832210730,NFP,SAN FRANCISCO,CA
7,"10/40 Connections, Inc.",45-70-15,621825230,NFP,HIXSON,TN
8,"1000 Feet Project, Inc",45-00-14,473820859,NFP,NEW YORK,NY
9,1000 Islands Hose Haulers,45-38-38,454570241,NFP,CARTHAGE,NY


In [32]:
###LOAD THE FILE INTO S3####
# prepare csv file name   
pathname = 's3:/alejandrabucket2/'#specify location of s3:/{my-bucket}/
filename= 'all_charities_bureau_scrape_data' #name of your group
datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
filenames3 = "%s%s%s.csv"%(pathname,filename,datetime) #name of the filepath and csv file

#load file into s3. Pandas actually leverages boto to connect to s3 and can push the file directly into an s3 bucket
table_1.to_csv(filenames3, header=True, line_terminator='\n') 

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

Successfull uploaded file to location:s3:/alejandrabucket2/all_charities_bureau_scrape_data20210505234835.csv
